In [1]:
import math
import numpy as npy
import EMfields as em
from scipy import special as sp

In [ ]:
def Esqrmod(wl,N2,N,r,theta,phi,a,mu1,isSphere):
    if isSphere:
        E, H = em.sphereEM(wl,N2,N,r,theta,phi,a,mu1)
    else:
        E, H = em.shellEM(wl,N2,N,r,theta,phi,a,mu1)
    return E*npy.conj(E)

In [ ]:
def diff_volume(wl,N2,N,r,theta,phi,a,mu1,isSphere):
    Esqr = Esqrmod(wl,N2,N,r,theta,phi,a,mu1,isSphere)
    return Esqr*(r**2)*npy.sin(theta)

In [ ]:
def heatpowerDensity(wl,N2,N,r,theta,phi,a,mu1):
    wl /= 1e9
    eps0 = 8.8541878128e-12
    Esqr = Esqrmod(wl,N2,N,r,theta,phi,a,mu1,isSphere)
    return omega*eps0*((N2**2).imag)*Esqr

In [ ]:
def powerDensity(wl,N2,N,r,theta,phi,a,mu1):
    wl /= 1e9
    eps0 = 8.8541878128e-12
    Esqr = Esqrmod(wl,N2,N,r,theta,phi,a,mu1,isSphere)
    return omega*eps0*((N2**2).imag)*Esqr